# Timeseries Workflow

## setup

In addition to `quest` the following packages will also need to be installed in the environment to run this notebook:
* hvplot
* panel

These packages can be installed with the following command:
```
conda install -c pyviz -c conda-forge hvplot panel
```

In [ ]:
import quest
import hvplot.pandas
import panel as pn

# Query USGS Data Service for Data

In [ ]:
service_selector = quest.util.ServiceSelector(default='svc://usgs-nwis:iv')
pn.panel(service_selector)

In [ ]:
service = service_selector.value
download_options = quest.api.get_download_options(service, fmt='param')[service]
download_options.parameter = 'streamflow'  # get streamflow data so we can calculate a flow duration curve
pn.panel(download_options)

In [ ]:
dataset = quest.api.get_data(
    service_uri=service,
    search_filters={'bbox': [-91, 32.3, -90.8, 32.34]},
    download_options=download_options,
    collection_name='examples',
    use_cache=True,
    as_open_datasets=False,
    raise_on_error=True,
)[0]
data = quest.api.open_dataset(dataset)
data.head()

In [ ]:
data[download_options.parameter].hvplot()

# Run Tools to Transform Downloaded Data

In [ ]:
tool = quest.tools.ts_resample
tool.dataset = dataset
pn.panel(tool)

In [ ]:
new_dataset = tool()['datasets'][0]
new_data = quest.api.open_dataset(new_dataset)
new_data.index = new_data.index.to_timestamp()
new_data.head()

In [ ]:
parameter = quest.api.get_metadata(new_dataset)[new_dataset]['parameter']
new_data[parameter].hvplot()

In [ ]:
tool = quest.tools.flow_duration
tool.dataset = new_dataset
pn.panel(tool)

In [ ]:
new_dataset = tool()['datasets'][0]
new_data = quest.api.open_dataset(new_dataset)
parameter = quest.api.get_metadata(new_dataset)[new_dataset]['parameter']
new_data[parameter].hvplot()